<a href="https://colab.research.google.com/github/rvishakhs/Deep_Learning/blob/Fundamentals/Transfer_learning_with_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [6]:
!wget https://raw.githubusercontent.com/rvishakhs/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py


--2025-01-29 13:37:40--  https://raw.githubusercontent.com/rvishakhs/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-01-29 13:37:41 (89.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [7]:
# @title Default title text
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [8]:
# Get the data Ready
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip


--2025-01-29 13:37:55--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.184.207, 74.125.126.207, 74.125.132.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.184.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   176MB/s    in 0.9s    

2025-01-29 13:37:56 (176 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [9]:
unzip_data('10_food_classes_10_percent.zip')

In [10]:
walk_through_dir('/content/10_food_classes_10_percent')

There are 2 directories and 0 images in '/content/10_food_classes_10_percent'.
There are 10 directories and 0 images in '/content/10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '/content/10_food_cl

In [11]:
# Create training and test directory paths

train_dir = '/content/10_food_classes_10_percent/train'
test_dir = '/content/10_food_classes_10_percent/test'

In [12]:
import tensorflow as tf
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=IMG_SIZE,
                                                                 batch_size=BATCH_SIZE,
                                                                 label_mode='categorical')
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                 image_size=IMG_SIZE,
                                                                 batch_size=BATCH_SIZE,
                                                                 label_mode='categorical')

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [13]:
train_data


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [ ]:
# Building a transfer learning model with functional api tensorflow
# 1. Create base model with tf.keras.applications
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# 2. Free the base model ( so the underlying pre-trained patterns aren't updated)
base_model.trainable = False

# 3. Create inputs into our model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='Input_layer')

# 4. Pass the inputs to the base_model
x = base_model(inputs)
print(f'Shape after passing inputs through the base model: {x.shape}')

# 5. Average pool the outputs of the basemodel (aggregate ball the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x)
print(f'Shape after GlobalAveragePooling2D: {x.shape}')

# 6. Create the output activation layer
outputs = tf.keras.layers.Dense(10, activation='softmax', name='output_layer')(x)

# 7. Combine the inputs with the output into a model
model_0 = tf.keras.Model(inputs, outputs)

# 9. Compile for the model
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy']
                )

history_0 = model_0.fit(train_data,
            epochs=5,
            steps_per_epoch=len(train_data) // 32,
            validation_data=test_data,
            validation_steps=int(0.25 * len(test_data) // 32),
            callbacks=[create_tensorboard_callback(dir_name='transfer_learning',
                                                   experiment_name='10_percentage_data')])

In [ ]:
model_0.summary()

In [ ]:
# check out our model loss function
plot_loss_curves(history_0)

In [ ]:
# Model 2 fetaure extraction transfer learning and data augmentation in place
# Get the data Ready
#!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

In [ ]:
# Seting up train and test dir
train_dir_aug = '/content/10_food_classes_10_percent/train'
test_dir_aug = '/content/10_food_classes_10_percent/test'

In [ ]:
# Set data inputs

import tensorflow as tf
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_data_10_percentage = tf.keras.preprocessing.image_dataset_from_directory(train_dir_aug,
                                                                               label_mode='categorical',
                                                                               batch_size=BATCH_SIZE,
                                                                               image_size=IMG_SIZE)
test_data_10_percentage = tf.keras.preprocessing.image_dataset_from_directory(test_dir_aug,
                                                                               label_mode='categorical',
                                                                               batch_size=BATCH_SIZE,
                                                                               image_size=IMG_SIZE)

In [ ]:
# Creatinig a data augmentation layer with seqential api

from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

data_augmentation = Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomHeight(0.2),
    layers.RandomWidth(0.2),
    layers.RandomZoom(0.2),
    layers.RandomRotation(0.2)
   #Layers.Rescaling(1./255) #if we are using ResNet or other models which requires scaled values
], name='data_augmentation')



In [ ]:
# setup the input shape to our model
input_shape = (224, 224, 3)

# Create a frozen basemodel
base_model = tf.keras.applications.EfficientNetB1(include_top=False)
base_model.trainable = False

# Create the input and output layers (include the layers between)
inputs = layers.Input(shape=input_shape, name='input_layer')

# Passing the input layer to the data augmentation layer we created above
x = data_augmentation(inputs)  #Augmenting our training images

# Passing the augmented data into our base model and keep the training mode off
x = base_model(x, training=False)

# Passing to the pooling layer for feature extraction
x = layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x)

# Creating the output layer and passing the x to it
outputs = layers.Dense(10, activation='softmax', name='output_layer')(x)

# Build the model
model_2 = tf.keras.Model(inputs, outputs)

# Compile the model
model_2.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

# Fit the model
history_2 = model_2.fit(train_data_10_percentage,
                        epochs=5,
                        steps_per_epoch= len(train_data_10_percentage) // 32,
                        validation_data=test_data_10_percentage,
                        validation_steps=int(0.25 * len(test_data_10_percentage) // 32),
                        callbacks=[create_tensorboard_callback(dir_name='transfer_learning',
                                                   experiment_name='10_percentage_data_augmented')])

In [ ]:
# Create a model checkpoint call back
checkpoint_path = 'ten_percentage_model_checkpoint/checkpoint.ckpt'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_best_only=False,
                                                         save_weights_only=True,
                                                         save_freq='epoch',
                                                         verbose=1)
